In [ ]:
from dataset.OfflineDataset import OfflineDataset
import torch
from tensordict import TensorDict
from torch.utils.data import DataLoader, RandomSampler

In [206]:
td = TensorDict({
    'soe': torch.randint(0,10,(17520,1), dtype=torch.float32),
    'action': torch.randint(-1,1,(17520,1), dtype=torch.float32),
    'prosumption': torch.randint(-2,2,(17520,1), dtype=torch.float32),
    'prosumption_forecast': torch.randint(-2,2,(17520,5), dtype=torch.float32),
    'observation': torch.randint(-2,2,(17520,10), dtype=torch.float32),
    
}, batch_size=torch.Size([17520,]),
device='cpu'
)
torch.save(td, '../output/1.pt')

In [207]:
td

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([17520, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        observation: Tensor(shape=torch.Size([17520, 10]), device=cpu, dtype=torch.float32, is_shared=False),
        prosumption: Tensor(shape=torch.Size([17520, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        prosumption_forecast: Tensor(shape=torch.Size([17520, 5]), device=cpu, dtype=torch.float32, is_shared=False),
        soe: Tensor(shape=torch.Size([17520, 1]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([17520]),
    device=cpu,
    is_shared=False)

In [45]:
dataset = OfflineDataset('../output',1)
len(dataset)

5

In [87]:
dataset[0]

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False),
        prosumption: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False),
        prosumption_forecast: Tensor(shape=torch.Size([2]), device=cpu, dtype=torch.float32, is_shared=False),
        soe: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([]),
    device=None,
    is_shared=False)

In [77]:
dataloader = iter(DataLoader(dataset=td, batch_size=1, shuffle=True, collate_fn=lambda x: x))

In [83]:
dataloader = iter(DataLoader(dataset=dataset, batch_size=1, shuffle=True))

In [213]:
import numpy as np

In [226]:
class OffData():
    def __init__(self, generated_data_path, batch_size, ep_len):
        self._td = torch.load(generated_data_path, weights_only=False)
        self._batch_size = batch_size
        self._ep_len = ep_len
        self._num_traj = int(td.batch_size.numel() / ep_len)

    def __len__(self):
        return self._num_traj
    
    def sample(self):
        inds = torch.randint(low=0, high=self._num_traj-1, size=(self._batch_size,))
        
        s, a, r, rtg, t = [],[],[],[],[]
        for i in inds:
            traj = self._td[i:i+self._ep_len]
            s.append(traj['observation'].unsqueeze(0))
            a.append(traj['action'].unsqueeze(0))
            # r.append(traj['next','observation'].unsqueeze(-1))
            t.append(np.arange(0,self._ep_len))
        states = torch.cat(s, dim=0)
        actions = torch.cat(a, dim=0)
        # rewards = torch.cat(r, dim=0)
        timesteps = torch.from_numpy(np.concatenate(t, axis=0))

        return states, actions, timesteps


In [255]:
OffData('../output/1.pt', 1,48).sample()

(tensor([[[-2.,  0.,  1.,  1.,  0., -2.,  1.,  0., -2.,  0.],
          [ 1., -1.,  1.,  1., -2.,  0., -2.,  0.,  1.,  1.],
          [ 0., -1.,  0.,  0., -1.,  0., -2., -1.,  1.,  1.],
          [ 0., -2.,  0.,  0., -2., -1.,  1., -1., -2.,  1.],
          [ 0., -2., -1., -2., -2., -1.,  1., -1., -1.,  1.],
          [ 0., -2.,  0.,  0., -1., -2., -2.,  0., -1.,  0.],
          [-2., -2.,  1.,  1., -2.,  1., -1., -2.,  1., -1.],
          [-1., -1., -1., -1., -2.,  1.,  0.,  0.,  0.,  0.],
          [-2., -2., -1.,  0., -1., -1., -1.,  1., -1.,  1.],
          [ 1., -2.,  0.,  0., -1., -1., -2.,  1.,  1., -1.],
          [ 1.,  1., -2., -1.,  0.,  0., -2.,  0.,  1., -1.],
          [-1., -2., -1.,  0., -2.,  1., -1., -1., -1., -1.],
          [ 0., -1., -1., -1.,  0.,  0.,  1.,  0., -2., -1.],
          [-1.,  1.,  1., -1.,  1.,  0.,  1.,  0., -1., -2.],
          [-1., -2.,  1., -2.,  0.,  0., -1., -1.,  0., -1.],
          [ 0., -1., -2., -1., -2.,  1., -2., -2.,  0., -1.],
        